In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
%%capture

!pip install faiss-gpu -q

!pip install deep-translator -q

!pip install langdetect -q

!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip uninstall -y torch torchvision torchaudio xformers
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install sentence_transformers peft

!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [5]:
import torch

import numpy as np

import matplotlib.pyplot as plt

import transformers

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from sentence_transformers import SentenceTransformer

import faiss

from torch.utils.data import Dataset, DataLoader

import torch.nn as nn

from nltk import sent_tokenize

import nltk

from deep_translator import GoogleTranslator

from langdetect import detect

import pickle

import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from peft import prepare_model_for_kbit_training
from peft import LoraConfig, TaskType, get_peft_model

from sklearn.model_selection import train_test_split

from unsloth import FastLanguageModel

import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
!rm -rf prikols
!git clone https://github.com/bivba/prikols

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'prikols'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 47 (delta 13), reused 14 (delta 5), pack-reused 17 (from 1)
Receiving objects: 100% (47/47), 42.61 MiB | 32.13 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [7]:
token = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

device = 'cuda' if torch.cuda.is_available() else 'cpu'



client = GoogleTranslator(source='ru', target='en')

text = open('/kaggle/working/prikols/context.txt').read()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
#model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"

torch.cuda.empty_cache()

max_seq_length = 2048

dtype = None

load_in_4bit = True



model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "bivba/quant_4bit",

    max_seq_length = max_seq_length,

    dtype = dtype,

    load_in_4bit = load_in_4bit

)

==((====))==  Unsloth 2024.12.4: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/210k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/275M [00:00<?, ?B/s]

Unsloth 2024.12.4 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [9]:
nltk.download('punkt_tab')

def split_document(document, chunk_size=256, overlap=32):

    sentences = sent_tokenize(document)

    chunks = []

    current_chunk = []

    current_length = 0



    for sentence in sentences:

        sentence_length = len(sentence)

        if current_length + sentence_length <= chunk_size:

            current_chunk.append(sentence)

            current_length += sentence_length

        else:

            chunks.append(" ".join(current_chunk))

            current_chunk = current_chunk[-overlap:] + [sentence]

            current_length = sum(len(s) for s in current_chunk)



    if current_chunk:

        chunks.append(" ".join(current_chunk))



    return chunks

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [10]:
chunks = split_document(text)

encoded = token.encode(chunks, return_numpy=True)

Batches:   0%|          | 0/486 [00:00<?, ?it/s]

In [11]:
faiss_index = faiss.read_index('prikols/context (1).faiss')

def find_context(query, k=3):

  if detect(query) == 'ru':

    query = client.translate(query)

  q_emb = token.encode([query])

  q_emb = q_emb.astype(np.float32)

  _, indices = faiss_index.search(np.array(q_emb), k)

  retrieved_chunks = [chunks[i] for i in indices[0]]

  context = ' '.join(retrieved_chunks)

  return context

In [12]:
question_prompt = """You are an AI assistaint helping with answering questions about quantum physics and quantum computer science. Below is a Question with a supportive context. Write a response that appropriately answers the request.



### Question:

{}



### Context:

{}



### Answer:

{}"""



EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):

    instructions = [client.translate(example) for example in examples['quesion']] if (all([detect(example) for example in examples['question']]) == 'ru') else examples['question']
    inputs       = [find_context(instruction) for instruction in instructions]
    outputs      = [client.translate(example) for example in examples['answer']] if (all([detect(example) for example in examples['answer']]) == 'ru') else examples['answer']

    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):

        # Must add EOS_TOKEN, otherwise your generation will go on forever!

        text = question_prompt.format(instruction, input, output) + EOS_TOKEN

        texts.append(text)

    return { "text" : texts, }

pass


In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(

[

    question_prompt.format(

        client.translate(" Кубит - это суперпозиция двух квантовых состояний $c_0|0> + c_1|1>$, где $c_1, c_2$ - комплексные числа. Сколько действительных параметров нужно задать, чтобы определить конкретный кубит. Выбери один из вариантов \n 1) 2 \n 2) 3 \n 3) 1 \n 4) 4 \n"), # instruction

        find_context(" Кубит - это суперпозиция двух квантовых состояний $c_0|0> + c_1|1>$, где $c_1, c_2$ - комплексные числа. Сколько действительных параметров нужно задать, чтобы определить конкретный кубит. \\\\\n1) 2 \\\\\n2) 3 \\\\\n3) 1 \\\\\n4) 4 \\\\\n"), # input

        "", # output - leave this blank for generation!

    )

], return_tensors = "pt").to("cuda")



outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, pad_token_id = tokenizer.eos_token_id)

tokenizer.batch_decode(outputs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

["You are an AI assistaint helping with answering questions about quantum physics and quantum computer science. Below is a Question with a supportive context. Write a response that appropriately answers the request.\n\n\n\n### Question:\n\nA qubit is a superposition of two quantum states $c_0|0> + c_1|1>$, where $c_1, c_2$ are complex numbers. How many real parameters must be specified to define a specific qubit. Choose one of the options \n1) 2 \n2) 3 \n3) 1 \n4) 4\n\n\n\n### Context:\n\n[label=0., ref=0]\n2. **General Two-Qubit States** In our system of two qubits, the state space is four-dimensional with basis states \\(\\ket{00}\\), \\(\\ket{01}\\), \\(\\ket{10}\\), and \\(\\ket{11}\\). This allows us to construct a general two-qubit state as a linear combination of these basis states: \\[\\ket{\\psi}=\\sum_{i,j\\in\\{0,1\\}}c_{ij}\\ket{ij}=c_{00}\\ket{00}+c_{01}\\ket{01}+c_ {10}\\ket{10}+c_{11}\\ket{11}.\\] (6.14)\n\nHere, \\(c_{ij}\\) are complex coefficients such that \\(\\sum|c